# Candle API Examples

---

## Install Package

In [ ]:
%pip install .. ipywidgets

In [ ]:
import candle_api
import threading
import ipywidgets as widgets
import time

## Find Device

In [ ]:
devs = candle_api.list_device()

if devs:
    sel = widgets.Dropdown(
        options=[d.serial_number for d in devs],
        value=devs[0].serial_number,
        disabled=False
    )
    inf = widgets.Output()

    def update_inf(s):
        global dev
        dev = next(d for d in devs if d.serial_number == s)
        inf.clear_output()
        with inf:
            print(f'vendor_id: {dev.vendor_id:04X}')
            print(f'product_id: {dev.product_id:04X}')
            print(f'manufacturer: {dev.manufacturer}')
            print(f'product: {dev.product}')
            print(f'serial_number: {dev.serial_number}')

    def on_sel_change(change):
        update_inf(change.new)

    update_inf(devs[0].serial_number)
    sel.observe(on_sel_change, names='value')
    display(widgets.VBox([sel, inf]))
else:
    print('No devices found')

## Open Device

In [ ]:
dev.open()

## Open Channel

In [ ]:
ch = dev[0]
ch.start(candle_api.CandleMode(listen_only=True, loop_back=True, fd=ch.feature.fd))

## Send and Receive

In [ ]:
running = True

def polling_cb():
    rx_cnt = 0
    tx_cnt = 0
    err_cnt = 0
    et = st = time.time()
    try:
        while running:
            et = time.time()
            frame = ch.receive(1)
            if frame.frame_type.error_frame:
                err_cnt += 1
            else:
                if frame.frame_type.rx:
                    rx_cnt += 1
                else:
                    tx_cnt += 1
    except TimeoutError:
        pass
    print(f'tx: {tx_cnt}, rx {rx_cnt}, err: {err_cnt}, dt: {et - st} s')

polling_thread = threading.Thread(target=polling_cb)
polling_thread.start()

for i in range(200000):
    ch.send(candle_api.CandleCanFrame(candle_api.CandleFrameType(), 0x123, 8, b'12345678'), 1)

time.sleep(1)
running = False
polling_thread.join()

## Close Device

In [ ]:
dev.close()